In [101]:
import numpy as np
import pandas as pd

import sqlite3

# 1. Lưu và đọc DataFrame vào/từ database

In [98]:
df_hocvien = pd.DataFrame({'name':['Lan', 'Mai', 'Phong'], 'age':[24, 22, 25]})
#df_test['id'] = df_test.index
df_hocvien

,name,age
0,Lan,24
1,Mai,22
2,Phong,25


In [102]:
# Kết nối đến CSDL
conn = sqlite3.connect('database/quan_ly_dao_tao.db')

In [105]:
df_hocvien.to_sql(name='hocvien', con=conn, if_exists='replace')
conn.commit()

In [106]:
pd.read_sql("SELECT * from hocvien", conn)

,index,name,age
0,0,Lan,24
1,1,Mai,22
2,2,Phong,25


In [108]:
df_monhoc = pd.DataFrame({'name':['Lập Trình Python', 'Machine Learning', 'Deep Learning'], 
                          'weeks':[10, 12, 12]})
df_monhoc

,name,weeks
0,Lập Trình Python,10
1,Machine Learning,12
2,Deep Learning,12


In [109]:
df_monhoc.to_sql(name='monhoc', con=conn, if_exists='replace')
conn.commit()

In [110]:
pd.read_sql("SELECT * from monhoc", conn)

,index,name,weeks
0,0,Lập Trình Python,10
1,1,Machine Learning,12
2,2,Deep Learning,12


# 2. Kết hợp pandas và SQL cho truy vấn dữ liệu

## 2.1. Sử dụng hàm df.query()

In [46]:
df_product = pd.read_csv('data/products.csv', index_col='product_id')
df_product.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


In [47]:
df_product.query("aisle_id == 45")

,product_name,aisle_id,department_id
product_id,,,
124,Sun Cups Dark Chocolate,45,19
136,Simple,45,19
184,Kisses Milk Chocolate Candy,45,19
207,Minis Candy Bars,45,19
227,Triple Chocolate Snack Size,45,19
...,...,...,...
49504,Dark Chocolate Peanut Butter Cups Miniatures,45,19
49601,Pomegranate Gummy Bears,45,19
49620,Milk Chocolate Mini's,45,19


In [48]:
df_product.query("aisle_id < department_id/5 and aisle_id > department_id/7")

,product_name,aisle_id,department_id
product_id,,,
25,Salted Caramel Lean Protein & Fiber Bar,3,19
77,Coconut Chocolate Chip Energy Bar,3,19
164,Peanut Butter Cacao Quinoa Clusters,3,19
183,Cookie Dough Performance Energy,3,19
205,Chocolate Mint Cookie Crunch Nutrition Bar,3,19
...,...,...,...
49415,All Natural Cookies & Cream Muscle Brownie,3,19
49525,Cinnamon Roll Flavor Protein Bar,3,19
49561,Crunchy Peanut Butter Bars,3,19


## 2.2. Sử dụng module `pandasql`

In [ ]:
%pip install -U pandasql

In [31]:
import pandasql as pdsql
from pandasql import sqldf
from pandasql import PandaSQL
print(pdsql)
print(sqldf)

<module 'pandasql' from '/home/tqkhai/Working/Tools/anaconda3/lib/python3.8/site-packages/pandasql/__init__.py'>
<function sqldf at 0x7f982048a820>


In [49]:
pdsql.sqldf("SELECT * FROM df_product WHERE aisle_id=3 LIMIT 10 OFFSET 5", locals())

,product_id,product_name,aisle_id,department_id
0,324,Strong & Kind Roasted Jalapeno Almond Protein Bar,3,19
1,332,Raspberry Fig Bars,3,19
2,370,Chocolate Caramel Whey Protein Bar,3,19
3,387,Chocolate Brownie Flavor Protein Bar,3,19
4,406,Vanilla Almond Protein Bar,3,19
5,445,Almond Honey Protein+ Bar,3,19
6,594,Gluten Free Peanut Butter Berry Chewy With A ...,3,19
7,657,"Nutrition Bar, Raw Organic, Pineapple Almond +...",3,19
8,701,Blueberry Blast Bar,3,19
9,729,Superfood Bar Goji Acai Organic,3,19


**Được khuyến cáo nên sử dụng lớp PandaSQL**

In [36]:
pdsql2 = PandaSQL()

In [50]:
pdsql2("SELECT aisle_id, product_id FROM df_product WHERE department_id=3 LIMIT 10 OFFSET 5", locals())

,aisle_id,product_id
0,93,131
1,112,189
2,8,218
3,93,245
4,112,317
5,112,346
6,112,444
7,43,517
8,112,566
9,128,595


In [51]:
pdsql2("SELECT aisle_id, product_id FROM df_product WHERE department_id=3 GROUP BY aisle_id", globals())

,aisle_id,product_id
0,8,218
1,43,517
2,93,116
3,112,101
4,128,58


## 2.3. Truy vấn từ nhiều dataframe

In [52]:
df_aisle = pd.read_csv('data/aisles.csv', index_col='aisle_id')
df_aisle.head()

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation


**Lấy ra 10 aisle có nhiều loại sản phẩm nhất**

In [111]:
query = """ SELECT a.aisle, COUNT(p.product_name) AS NProducts
            FROM df_product AS p
            LEFT JOIN  df_aisle a
            ON a.aisle_id = p.aisle_id
            GROUP BY a.aisle
            ORDER BY NProducts Desc
            LIMIT 10 OFFSET 1
            """
pdsql2(query, globals())

,aisle,NProducts
0,candy chocolate,1246
1,ice cream ice,1091
2,vitamins supplements,1038
3,yogurt,1026
4,chips pretzels,989
5,tea,894
6,packaged cheese,891
7,frozen meals,880
8,cookies cakes,874
9,energy granola bars,832


**Lấy tên của các sản phẩm và aisle tương ứng cho department số 3**

In [86]:
query = """ SELECT p.product_name, a.aisle
            FROM df_product p
            INNER JOIN df_aisle a
            ON p.aisle_id = a.aisle_id
            WHERE p.department_id = 3
            """
pdsql2(query, globals())

,product_name,aisle
0,Whole Wheat Tortillas,tortillas flat bread
1,100% Whole Wheat Pita Bread,tortillas flat bread
2,"Bread, Healthy Whole Grain",bread
3,Organic Sprouted Barley Bread,bread
4,English Muffins,breakfast bakery
...,...,...
1511,Gluten Free Classic Sourdough,bread
1512,Challah Bread,bread
1513,Sourdough Loaf,bread
1514,Masada Kosher Pocket Bread,tortillas flat bread


**Lấy ra tên của các department tương ứng**

In [59]:
df_department = pd.read_csv('data/departments.csv', index_col='department_id')
df_department.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


In [93]:
query = """ SELECT p.product_name, a.aisle, d.department
            FROM df_product p
            JOIN df_aisle a
                ON p.aisle_id = a.aisle_id
            JOIN df_department d
                ON d.department_id = p.department_id
            """
pdsql2(query, globals())

,product_name,aisle,department
0,Chocolate Sandwich Cookies,cookies cakes,snacks
1,All-Seasons Salt,spices seasonings,pantry
2,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,Green Chile Anytime Sauce,marinades meat preparation,pantry
...,...,...,...
49683,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol
49684,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen
49685,Artisan Baguette,bread,bakery
49686,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets
